In [ ]:
# default_exp parser

In [ ]:
#hide
import sys, os
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

%load_ext autoreload
%autoreload 2    

# Parser

In [ ]:
#export
from collections import defaultdict
from functools import partial
from fastcore.basics import uniqueify
from fastcore.utils import parallel
from fastcore.foundation import L
from rfpy.blocks import *
from rfpy.utils import bin2int, bin2str, optimize, getattrs
from rfpy.constants import *
from rfpy.cyparser import cy_extract_compressed
import pandas as pd
import numpy as np
import os
from pathlib import Path
from typing import *
from datetime import datetime

path_type = Union[str, Any]
bin_val = Union[int, bytes]
bytes_encoded = Union[int, bytes]
datetime_object = datetime

Fonte: # https://github.com/fastai/fastai/blob/master/fastai/data/transforms.py#L26

In [ ]:
#export
def _get_files(p, fs, extensions=None):
    p = Path(p)
    res = [p/f for f in fs if not f.startswith('.')
           and ((not extensions) or f'.{f.split(".")[-1].lower()}' in extensions)]
    return res

def get_files(path, extensions=None, recurse=True, folders=None, followlinks=True):
    "Get all the files in `path` with optional `extensions`, optionally with `recurse`, only in `folders`, if specified."
    path = Path(path)
    folders=L(folders)
    if extensions is not None:
        extensions = set(uniqueify(extensions))
        extensions = {e.lower() for e in extensions}
    if recurse:
        res = []
        for i,(p,d,f) in enumerate(os.walk(path, followlinks=followlinks)): # returns (dirpath, dirnames, filenames)
            if len(folders) !=0 and i==0: d[:] = [o for o in d if o in folders]
            else:                         d[:] = [o for o in d if not o.startswith('.')]
            if len(folders) !=0 and i==0 and '.' not in folders: continue
            res += _get_files(p, f, extensions)
    else:
        f = [o.name for o in os.scandir(path) if o.is_file()]
        res = _get_files(path, f, extensions)
    return L(res)

In [ ]:
#export
def parse_bin(bin_file, bytes_header: int = BYTES_HEADER, marker: bytes = ENDMARKER, slice_=None):
    with open(bin_file, mode='rb') as bfile:
        # O primeiro bloco do arquivo é o cabeçalho e tem 36 bytes de tamanho.
        header = bfile.read(bytes_header)
        body = bfile.read()
    if slice_ is not None:
        assert slice_.start >= bytes_header, f"The start of your slice has to be >= {bytes_header}, you passed {slice_.start} "
        body = body[slice_]
    return {'file_version': bin2int(header[:4]), 'string': bin2str(header[4:]), 'blocks': classify_blocks(L(body.split(marker)))}

```python
# Python >=3.8 only
def binary_iter(bin_file: path_type, marker: bytes = b'\x00UUUU', block_size: int = 4096) -> Iterator[bytes]:
    """
    str, bytes, int > bytes
    :param bin_file: arquivo binario que contém os dados
    :param marker: separador de blocos
    :param block_size: tamanho em bytes que é "lido" por vez no arquivo, evitando problemas de memória
    :return: bloco em formato binario
    Gerador que fornece a partir de de um arquivo binário, um bloco binário por vez.

    """
    with open(bin_file, mode='rb') as bfile:
        # O primeiro bloco do arquivo é o cabeçalho e tem 36 bytes de tamanho.
        yield bfile.read(36)
        # As demais partes podem prosseguir normalmente
        current = b''
        while block := bfile.read(block_size):
            current += block
            while (markerpos := current.find(marker)) > 0:
                yield current[:markerpos]
                current = current[markerpos + len(marker):]
```

A função a seguir mapeia o arquivo `.bin` nos devidos blocos

In [ ]:
#export
def classify_blocks(blocks: L)->Mapping[int,L]:
    """Receives an iterable L with binary blocks and returns a defaultdict with a tuple (block types, thread_id) as keys and a list of the Class Blocks as values
        :param file: A string or pathlib.Path like path to a `.bin`file generated by CFRS - Logger
        :return: A Dictionary with block types as keys and a list of the Class Blocks available as values
    """
    map_block = defaultdict(L)
    index = BYTES_HEADER
    for bloco in blocks:
        bloco = create_base_block(bloco)
        btype, btid = bloco.type, bloco.thread_id
        if btype == btid == 0: #Ignore empty block at the beginning if present
            continue
        stop = index + DATA_BLOCK_HEADER + len(bloco.data) + CHECKSUM
        map_block[(btype, btid)].append(((index,stop), block_constructor(btype, bloco)))
        #map_block[(btype, btid)].append(block_constructor(btype, bloco))
        #map_block[btype].append(((index,stop), block_constructor(btype, bloco)))
        index = stop + LEN_MARKER
    return map_block

In [ ]:
#export
def decode_compressed_block(block):
    MIN = block.offset - 127.5
    RUN = 255
    ESC = 254
    src = block.data[block.start:block.stop]
    nsrc = len(src)
    thresh = block.thresh
    dest = np.full(block.norig, MIN, dtype=np.float16)
    i = 0
    j = 0
    while i < nsrc:
        ib = src[i] 
        i+=1
        if ib == RUN:
            nrun = src[i] 
            i+=1
            dest[j:j+nrun] = MIN + thresh/2.
            j+=nrun
        elif ib == ESC:
            # next value is literal
            dest[j] = MIN + src[i]/2.
            i+=1 ; j+=1
        else:
            # value
            dest[j] = MIN + ib/2.
            j+=1
    return dest

In [ ]:
#export
def run_length_decode5(dest, src, nsrc, thresh, MIN):
    i = 0
    j = 0
    while i < nsrc:
        ib = src[i] 
        i+=1
        if ib == RUN:
            nrun = src[i] 
            i+=1
            dest[j:j+nrun] = thresh #dest is now a numpy array
            j+=nrun
        elif ib == ESC:
            # next value is literal
            dest[j] = src[i]/2. + MIN
            i+=1 ; j+=1
        else:
            # value
            dest[j] = ib/2. + MIN
            j+=1  

In [ ]:
#export
def _extract_uncompressed(blocks: Iterable, rows: int, cols: int, MIN: float):
    levels = np.full((rows, cols), MIN, dtype=np.float16)
    for b, block in enumerate(blocks):
        levels[b] = block.block_data
    return levels    


def extract_levels(spectrum_blocks: Iterable)->pd.DataFrame:
    """Receives a mapping `spectrum_blocks` and returns the Matrix with the Levels as values, Frequencies as columns and Block Number as index.
       :param pivoted: If False, optionally returns an unpivoted version of the Matrix
    """
    assert len(spectrum_blocks), f"The spectrum block list is empty"
    spectrum_blocks = spectrum_blocks.itemgot(1)
    block = spectrum_blocks[0]
    assert block.type in (63, 64, 67, 68), f"The input blocks are not spectral blocks" 
    rows = len(spectrum_blocks)
    MIN = block.offset - 127.5
    if block.type in (63,67):
        cols = block.ndata
        frequencies = getattr(spectrum_blocks[0], 'frequencies')
        return pd.DataFrame(_extract_uncompressed(spectrum_blocks, rows, cols, MIN), columns=frequencies)
    else:
        cols = block.norig
        thresh = block.thresh - 1
        block_data = [b.block_data for b in spectrum_blocks] 
        frequencies = np.linspace(block.start_mega, block.stop_mega, num=cols)
        return pd.DataFrame(cy_extract_compressed(block_data, rows, cols, thresh, MIN), columns=frequencies)
#     if not pivoted:
#         unpivot = np.array([np.repeat(np.arange(levels.shape[1]), levels.shape[0]),
#                             np.tile(frequencies, levels.shape[0]),
#                             levels.flatten()]).T
#         if not dtypes:
#             dtypes = LEVELS
#         return pd.DataFrame(unpivot, columns=LEVELS.keys()).astype(dtypes)

In [ ]:
#export
def meta2df(meta_list: Iterable, optimize: bool = False)->pd.DataFrame:
    """Receives and Iterable `metalist` with metadata and converts it to a DataFrame"""
    df = pd.DataFrame(meta_list)
    dt_features = ['wallclock_datetime'] if 'wallclock_datetime' in df.columns else []
    if optimize:
        df = optimize(df, dt_features)
    if dt_features:
        df = df.set_index('wallclock_datetime')
    return df

In [ ]:
#export
def extract_metadata(map_blocks: Mapping[int,L], subset=MAIN_BLOCKS)->pd.DataFrame:
    """Receives a Mapping with the different `. bin` Blocks and extracts the metadata from them excluding spectral data.
    """
    metadata = {}
    file_version, string, blocks = map_blocks.values()
    for (tipo, tid), blocos in blocks.items():
        if tipo not in subset:
            continue
        df = meta2df(blocos.map(lambda item: {**{'start_byte': item[0][0], 'stop_byte': item[0][1]}, **getattrs(item[1])}))        
        df['file_version'] = file_version
        metadata[(tipo, tid)] = df
    return metadata

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_main.ipynb.
Converted 01_parser.ipynb.
Converted 02_utils.ipynb.
Converted 03_blocks.ipynb.
Converted 04_constants.ipynb.
Converted index.ipynb.
